In [1]:
import cv2
import mediapipe as mp
from ultralytics import YOLO

In [22]:
# YOLO 모델 로드
yolo_model = YOLO('D:\\project\\prjvenv\\runs\\detect\\human_fall_s30\\weights\\best.pt')
print("YOLO 모델 로드 완료")

# MediaPipe 설정
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils
print("MediaPipe 설정 완료")

# 원하는 랜드마크 정의
DESIRED_LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

# 동영상 파일 열기
video_path = r'D:\041.낙상사고 위험동작 영상-센서 쌍 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\영상\Y\SY\00723_H_D_SY_C1\00723_H_D_SY_C1.mp4'
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: 비디오 파일을 열 수 없습니다.")
    exit()
print("비디오 파일 열기 성공")

# 결과 동영상 저장을 위한 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_combined.mp4', fourcc, 60.0, (int(cap.get(3)), int(cap.get(4))))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("비디오 종료")
        break

    frame_count += 1
    print(f"프레임 {frame_count} 처리 중")

    # YOLO로 사람 감지
    results = yolo_model(frame)
    print(f"YOLO 감지 결과: {len(results[0].boxes)} 개체 감지됨")

    for r in results:
        boxes = r.boxes
        for box in boxes:
            if int(box.cls) == 0:  # 사람 클래스
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                print(f"사람 감지: 바운딩 박스 좌표 ({x1}, {y1}, {x2}, {y2})")
                
                # 바운딩 박스 추출
                person_image = frame[y1:y2, x1:x2]
                
                # MediaPipe로 포즈 추정
                results_pose = pose.process(cv2.cvtColor(person_image, cv2.COLOR_BGR2RGB))
                
                if results_pose.pose_landmarks:
                    print("포즈 랜드마크 감지됨")
                    for idx, landmark in enumerate(results_pose.pose_landmarks.landmark):
                        if idx in DESIRED_LANDMARKS:
                            cx = int(landmark.x * person_image.shape[1])
                            cy = int(landmark.y * person_image.shape[0])
                            global_x = x1 + cx
                            global_y = y1 + cy
                            cv2.circle(frame, (global_x, global_y), 5, (0, 255, 0), -1)
                            print(f"랜드마크 {idx}: ({global_x}, {global_y})")
                else:
                    print("포즈 랜드마크를 감지하지 못함")
                
                # 바운딩 박스 그리기
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 5)

    # 결과 표시
    cv2.namedWindow("YOLO + MediaPipe", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("YOLO + MediaPipe", 1920, 1080)
    cv2.imshow("YOLO + MediaPipe", frame)
    
    # 결과 동영상 저장
    out.write(frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("사용자에 의해 종료됨")
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("프로그램 종료")

YOLO 모델 로드 완료
MediaPipe 설정 완료
비디오 파일 열기 성공
프레임 1 처리 중

0: 384x640 1 Non_Fall, 3.0ms
Speed: 2.0ms preprocess, 3.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1871, 203, 2230, 972)
포즈 랜드마크 감지됨
랜드마크 0: (2127, 293)
랜드마크 11: (2183, 390)
랜드마크 12: (2033, 375)
랜드마크 15: (2199, 600)
랜드마크 16: (1933, 568)
랜드마크 23: (2130, 615)
랜드마크 24: (2047, 610)
랜드마크 25: (2110, 775)
랜드마크 26: (2032, 773)
랜드마크 27: (2100, 902)
랜드마크 28: (2036, 899)
프레임 2 처리 중

0: 384x640 1 Non_Fall, 4.0ms
Speed: 1.0ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
YOLO 감지 결과: 1 개체 감지됨
사람 감지: 바운딩 박스 좌표 (1887, 201, 2225, 972)
포즈 랜드마크 감지됨
랜드마크 0: (2129, 291)
랜드마크 11: (2190, 386)
랜드마크 12: (2039, 373)
랜드마크 15: (2199, 608)
랜드마크 16: (1946, 569)
랜드마크 23: (2136, 613)
랜드마크 24: (2052, 608)
랜드마크 25: (2114, 773)
랜드마크 26: (2040, 770)
랜드마크 27: (2104, 897)
랜드마크 28: (2044, 897)
프레임 3 처리 중

0: 384x640 1 Non_Fall, 3.0ms
Speed: 2.0ms preprocess, 3.0ms infer